# Submit RTC jobs for new granules

Install dependencies:

In [ ]:
# TODO remove asf-search pin when https://github.com/asfadmin/Discovery-asf_search/issues/204 is resolved
!pip install asf-search==6.0.2 hyp3-sdk

import asf_search
from hyp3_sdk import HyP3

Define search parameters and job specification:

In [ ]:
search_parameters = {
    "start": "2023-06-20T00:00:00Z",
    "end": "2023-07-24T00:00:00Z",
    "intersectsWith":
        "POLYGON((-110.7759 44.8543,-101.3998 44.8543,-101.3998 50.8183,-110.7759 50.8183,-110.7759 44.8543))",
    "platform": "S1",
    "processingLevel": "SLC",
}
job_specification = {
    "job_parameters": {
        "resolution": 30,
    },
    "job_type": "RTC_GAMMA",
    "name": "new-rtc-job"
}

Construct a list of unprocessed granules:

In [ ]:
hyp3 = HyP3(api_url='https://hyp3-test-api.asf.alaska.edu/')

previous_jobs = hyp3.find_jobs(
    name=job_specification['name'],
    job_type=job_specification['job_type'],
)
processed_granules = [job.job_parameters['granules'][0] for job in previous_jobs]

search_results = asf_search.search(**search_parameters)
search_results.raise_if_incomplete()

unprocessed_granules = [
    result for result in search_results if result.properties['sceneName'] not in processed_granules
]

Submit a new job for each unprocessed granule:

In [ ]:
from copy import deepcopy

def get_job_for_granule(granule: asf_search.ASFProduct) -> dict:
    job = deepcopy(job_specification)
    job['job_parameters']['granules'] = [granule.properties['sceneName']]
    return job


new_jobs = [get_job_for_granule(granule) for granule in unprocessed_granules]
print(f'Submitting {len(new_jobs)} jobs')
if new_jobs:
    hyp3.submit_prepared_jobs(new_jobs)